# Quickstart: Deploy Azure IoT Operations to an Arc-enabled Kubernetes cluster using Codespaces

<div class="alert alert-block alert-info">
<b>IMPORTANT:</b> Azure IoT Operations Preview – enabled by Azure Arc is currently in PREVIEW.
You shouldn't use this preview software in production environments.

See the [Supplemental Terms of Use for Microsoft Azure Previews](https://azure.microsoft.com/support/legal/preview-supplemental-terms/) for legal terms that apply to Azure features that are in beta, preview, or otherwise not yet released into general availability.</div>

In this quickstart, you will deploy a suite of IoT services to an Azure Arc-enabled Kubernetes cluster so that you can remotely manage your devices and workloads. Azure IoT Operations Preview is a digital operations suite of services that includes Azure IoT Orchestrator. This quickstart guides you through using Orchestrator to deploy these services to a Kubernetes cluster. At the end of the quickstart, you have a cluster that you can manage from the cloud that's generating sample data to use in the following quickstarts.

The services deployed in this quickstart include:

* [Azure IoT Orchestrator](../deploy-custom/overview-orchestrator.md)
* [Azure IoT MQ](../manage-mqtt-connectivity/overview-iot-mq.md)
* [Azure IoT OPC UA broker](../manage-devices-assets/overview-opcua-broker.md) with simulated thermostat asset to start generating data
* [Azure IoT Data Processor](../process-data/overview-data-processor.md) with a demo pipeline to start routing the simulated data
* [Azure IoT Akri](../manage-devices-assets/overview-akri.md)
* [Azure Device Registry](../manage-devices-assets/overview-manage-assets.md#manage-assets-as-azure-resources-in-a-centralized-registry)
* [Azure IoT Layered Network Management](../manage-layered-network/overview-layered-network.md)
* [Observability](../monitor/howto-configure-observability.md)

## Prerequisites

Review the prerequisites based on the environment you use to host the Kubernetes cluster.

For this quickstart, we will use GitHub Codespaces to get started in a virtual environment without installing new tools.

* An Azure subscription. If you don't have an Azure subscription, [create one for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F) before you begin.

* A [GitHub](https://github.com) account.

---
  
## What problem will we solve?

Azure IoT Operations is a suite of data services that run on Kubernetes clusters. You want these clusters to be managed remotely from the cloud, and able to securely communicate with cloud resources and endpoints. We address these concerns with the following tasks in this quickstart:

1. Connect a Kubernetes cluster to Azure Arc for remote management.
1. Create an Azure Key Vault to manage secrets for your cluster.
1. Configure your cluster with a secrets store and service principal to communicate with cloud resources.
1. Deploy Azure IoT Operations to your cluster.

## Connect a Kubernetes cluster to Azure Arc

Azure IoT Operations should work on any Kubernetes cluster that conforms to the Cloud Native Computing Foundation (CNCF) standards. For this quickstart, use GitHub Codespaces.

To connect your cluster to Azure Arc:

1. Sign in to Azure CLI on your development machine or in your codespace terminal. To prevent potential permission issues later, sign in interactively with a browser here even if you've already logged in before.

```azurecli
az login
```

<div class="warning" style='background-color:#E9D8FD; color: #69337A; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:left'>
<b>NOTE</b>
When using a Github codespace in a browser, <code>az login</code> returns a localhost error in the browser window after logging in. To fix, either:
<br>
* Open the codespace in VS Code desktop, and then run <code>az login</code> again in the browser terminal.
* After you get the localhost error on the browser, copy the URL from the browser and run <code>curl "copied URL"</code> in a new terminal tab. You should see a JSON response with the message "You have logged into Microsoft Azure!."
</span>
</div>

1. Set environment variables for the rest of the setup. You will be prompted to input your own values. A new cluster and resource group are created in your Azure subscription based on the names you provide:

In [ ]:

# Azure region where the created resource group will be located
# Currently supported regions: "eastus", "eastus2", "westus", "westus2", "westus3", "westeurope", or "northeurope"
$LOCATION="WestUS3"

$RESOURCE_GROUP = Read-Host -Prompt "Name of a new resource group to create which will hold the Arc-enabled cluster and Alice Springs resources"
Write-Host "Resource group: $RESOURCE_GROUP" -ForegroundColor Blue

$CLUSTER_NAME = Read-Host -Prompt "Name of the Arc-enabled cluster to create in your resource group"
Write-Host "Arc cluster name: $CLUSTER_NAME" -ForegroundColor Blue

Resource group: rg-emm12122
Arc cluster name: emm12122


1. Set the Azure subscription context for all commands:

In [ ]:
# Id of the subscription where your resource group and Arc-enabled cluster will be created

$SUBSCRIPTION_ID = (az account show --query id -o tsv)
az account set -s $SUBSCRIPTION_ID

> **OPTIONAL:** If you have more than one subscription, run the commands below to list them then set the <SUBSCRIPTION_ID> variable to the ID of the subscription you want to use.

In [ ]:
# Returns all subscriptions for your account

az account list --query "[].{Environment:name,SubscriptionId:id}" -o table

In [ ]:
# Id of the subscription where your resource group and Arc-enabled cluster will be created

$subscriptionId = "<SUBSCRIPTION_ID>"

In [ ]:
az login


1. Register the required resource providers in your subscription:

In [ ]:
az provider register -n "Microsoft.ExtendedLocation"
az provider register -n "Microsoft.Kubernetes"
az provider register -n "Microsoft.KubernetesConfiguration"
az provider register -n "Microsoft.IoTOperationsOrchestrator"
az provider register -n "Microsoft.IoTOperationsMQ"
az provider register -n "Microsoft.IoTOperationsDataProcessor"
az provider register -n "Microsoft.DeviceRegistry"

To check if the registration is complete, run the following command. If the registration is complete, the command returns `Registered`.

In [ ]:
az provider list --query "[?(namespace=='Microsoft.ExtendedLocation' || namespace=='Microsoft.Kubernetes' || namespace=='Microsoft.KubernetesConfiguration' || namespace=='Microsoft.IoTOperationsOrchestrator' || namespace=='Microsoft.IoTOperationsMQ' || namespace=='Microsoft.IoTOperationsDataProcessor' || namespace=='Microsoft.DeviceRegistry')].{Provider:namespace,Status:registrationState}" -o table

1. Use the [az group create](/cli/azure/group#az-group-create) command to create a resource group in your Azure subscription to store all the resources:

In [ ]:
az group create --location $LOCATION --resource-group $RESOURCE_GROUP --subscription $SUBSCRIPTION_ID

1. Use the [az connectedk8s connect](/cli/azure/connectedk8s#az-connectedk8s-connect) command to Arc-enable your Kubernetes cluster and manage it in the resource group you created in the previous step:

In [ ]:
az connectedk8s connect -n $CLUSTER_NAME -l $LOCATION -g $RESOURCE_GROUP --subscription $SUBSCRIPTION_ID

1. Use the [az connectedk8s enable-features](/cli/azure/connectedk8s#az-connectedk8s-enable-features) command to enable custom location support on your cluster. Use the `objectId` or `id` value from the previous command to enable custom locations on the cluster:

In [ ]:
$OBJECT_ID = (az ad sp show --id bc313c14-388c-4e7d-a58e-70017303ee3b --query id -o tsv)

az connectedk8s enable-features -n $CLUSTER_NAME -g $RESOURCE_GROUP --custom-locations-oid $OBJECT_ID --features cluster-connect custom-locations


---

## Configure cluster and deploy Azure IoT Operations

Part of the deployment process is to configure your cluster so that it can communicate securely with your Azure IoT Operations components and key vault. The Azure CLI command `az iot ops init` does this for you. Once your cluster is configured, then you can deploy Azure IoT Operations.

Use the Azure CLI to create a key vault, build the `az iot ops init` command based on your resources, and then deploy Azure IoT Operations components to your Arc-enabled Kubernetes cluster.

### Create a key vault

You can use an existing key vault for your secrets, but verify that the **Permission model** is set to **Vault access policy**. You can check this setting in the Azure portal in the **Access configuration** section of an existing key vault. Or use the [az keyvault show](/cli/azure/keyvault#az-keyvault-show) command to check that `enableRbacAuthorization` is false.

To create a new key vault, use the following command:

In [ ]:
$KEY_VAULT_NAME = Read-Host -Prompt "Name of the key vault to create in your resource group"

az keyvault create --enable-rbac-authorization false --name $KEY_VAULT_NAME --resource-group $RESOURCE_GROUP


### Deploy Azure IoT Operations

1. In the Azure portal search bar, search for and select **Azure Arc**.

1. Select **Azure IoT Operations (preview)** from the **Application services** section of the Azure Arc menu.

   ![Screenshot of selecting Azure IoT Operations from Azure Arc](./media/quickstart-deploy/arc-iot-operations.png)

1. Select **Create**.

1. On the **Basics** tab of the **Install Azure IoT Operations Arc Extension** page, provide the following information:

   | Field | Value |
   | ----- | ----- |
   | **Subscription** | Select the subscription that contains your Arc-enabled Kubernetes cluster. |
   | **Resource group** | Select the resource group that contains your Arc-enabled Kubernetes cluster. |
   | **Cluster name** | Select your cluster. When you do, the **Custom location** and **Deployment details** sections autofill. |

   ![Screenshot of the basics tab for installing the Azure IoT Operations Arc extension in the Azure portal](./media/quickstart-deploy/install-extension-basics.png)

1. Select **Next: Configuration**.

1. On the **Configuration** tab, provide the following information:

   | Field | Value |
   | ----- | ----- |
   | **Deploy a simulated PLC** | Switch this toggle to **Yes**. The simulated PLC creates demo telemetry data that you use in the following quickstarts. |
   | **Mode** | Set the MQ configuration mode to **Auto**. |

   ![Screenshot of the configuration tab for installing the Azure IoT Operations Arc extension in the Azure portal](./media/quickstart-deploy/install-extension-configuration.png)

1. Select **Next: Automation**.

1. On the **Automation** tab, select **Pick or create an Azure Key Vault**.

   ![Screenshot of selecting your key vault in the automation tab for installing the Azure IoT Operations Arc extension in the Azure portal](./media/quickstart-deploy/install-extension-automation-1.png)

1. Provide the following information to connect a key vault:

   | Field | Value |
   | ----- | ----- |
   | **Subscription** | Select the subscription that contains your Arc-enabled Kubernetes cluster. |
   | **Key vault** | Choose the key vault that you created in the previous section from the drop-down list. |

1. Select **Select**.

1. On the **Automation** tab, the automation commands are populated based on your chosen cluster and key vault. Copy the **Required** CLI command.

   ![Screenshot of copying the CLI command from the automation tab for installing the Azure IoT Operations Arc extension in the Azure portal](./media/quickstart-deploy/install-extension-automation-2.png)

1. Run the `az iot ops init` command in the next cell. This should be a copy of the command you copied in the previous step.

   >**TIP**
   >If you get an error that says *Your device is required to be managed to access your resource*, go back to the previous step and make sure that you signed in interactively.



In [ ]:
az iot ops init --subscription $SUBSCRIPTION_ID -g $RESOURCE_GROUP --cluster $CLUSTER_NAME --kv-id /subscriptions/$SUBSCRIPTION_ID/resourceGroups/$RESOURCE_GROUP/providers/Microsoft.KeyVault/vaults/$KEY_VAULT_NAME --custom-location emm12122-cl --target $CLUSTER_NAME-target --dp-instance $CLUSTER_NAME-processor --mq-instance mq-instance --mq-mode auto

In [ ]:
az iot ops init --subscription $SUBSCRIPTION_ID -g $RESOURCE_GROUP --cluster $CLUSTER_NAME --kv-id /subscriptions/$SUBSCRIPTION_ID/resourceGroups/$RESOURCE_GROUP/providers/Microsoft.KeyVault/vaults/$KEY_VAULT_NAME --no-deploy

If `az iot ops` does not work, then run the following:

In [ ]:
# Remove old extensions:
az extension remove --name azure-iot-ops
az extension remove --name azure-edge

# Install extension "0.1.0b1"
az extension add --source $(curl -w "%{url_effective}\n" -I -L -s -S https://aka.ms/aziotopscli-latest -o /dev/null) -y

1. Return to the Azure portal and select **Review + Create**.

1. Wait for the validation to pass and then select **Create**.

## View resources in your cluster

While the deployment is in progress, you can watch the resources being applied to your cluster. You can use kubectl commands to observe changes on the cluster or, since the cluster is Arc-enabled, you can use the Azure portal.

To view the pods on your cluster, run the following command:

In [ ]:
#!pwsh
kubectl get pods -n azure-iot-operations


It can take several minutes for the deployment to complete. Continue running the `get pods` command to refresh your view.

To view your cluster on the Azure portal, use the following steps:

1. In the Azure portal, navigate to the resource group that contains your cluster.

1. From the **Overview** of the resource group, select the name of your cluster.

1. On your cluster, select **Extensions** from the menu.

   You can see that your cluster is running extensions of the type **microsoft.iotoperations.x**, which is the group name for all of the Azure IoT Operations components and the orchestration service.

   There's also an extension called **akvsecretsprovider**. This extension is the secrets provider that you configured and installed on your cluster with the `az iot ops init` command. You might delete and reinstall the Azure IoT Operations components during testing, but keep the secrets provider extension on your cluster.

## How did we solve the problem?

In this quickstart, you configured your Arc-enabled Kubernetes cluster so that it could communicate securely with your Azure IoT Operations components. Then, you deployed those components to your cluster. For this test scenario, you have a single Kubernetes cluster that's probably running locally on your machine. In a production scenario, however, you can use the same steps to deploy workloads to many clusters across many sites.

## Clean up resources

If you're continuing on to the next quickstart, keep all of your resources.

If you want to delete the Azure IoT Operations deployment but plan on reinstalling it on your cluster, be sure to keep the secrets provider on your cluster. In your cluster on the Azure portal, select the extensions of the type **microsoft.iotoperations.x** and **microsoft.deviceregistry.assets**, then select **Uninstall**.

If you want to delete all of the resources you created for this quickstart, delete the Kubernetes cluster that you deployed Azure IoT Operations to and remove the Azure resource group that contained the cluster.

## Next steps

> [Quickstart: Add OPC UA assets to your Azure IoT Operations cluster](quickstart-virtual-add-assets.ipynb)